In [ ]:
import pandas as pd
import nltk
import os
from dotenv import load_dotenv
import typing
import fmpsdk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [1]:
def sbe_sentiment_analysis(apikey, ticker, limit=50):
    """
    IN: 
    apikey: This should be your personalized api key. You can get one here 
    (https://fmpcloud.io/)
    ticker: This is the stock symbol as or trade name. e.g. TSLA, AAPL, GOOG
    limit: This number references how far back in date you want to go. If you
    want to find the date range please use get_timeframe() on this methods returned df.
    
    OUT: Pandas Dataframe
    """
    
    
    
    news = fmpsdk.stock_news(apikey=apikey, tickers=[ticker], limit=limit)
    df = pd.DataFrame(columns=['Date', 'Title', 'Text', 'CompoundScoreTitle',
                          'PositveScoreTitle', 'NegativeScoreTitle', 
                          'CompoundScoreText', 'NeutralScoreTitle',
                           'NeutralScoreText', 'textLeans', 'titleLeans',
                          'PositveScoreText', 'NegativeScoreText',
                          'CompoundAvg'])
    
    dates = []
    titles = []
    text_ = []


    for article in news:
        dates.append(article['publishedDate'])
        titles.append(article['title'])
        text_.append(article['text']) 
    df.Date = dates
    df.Title = titles
    df.Text = text_
    

    sid = SentimentIntensityAnalyzer()
    
    df.CompoundScoreTitle = df.Title.apply(lambda title:
    sid.polarity_scores(title)['compound'])

    df.PositveScoreTitle = df.Title.apply(lambda title:
        sid.polarity_scores(title)['pos'])

    df.NegativeScoreTitle = df.Title.apply(lambda title:
        sid.polarity_scores(title)['neg'])

    df.NeutralScoreTitle = df.Title.apply(lambda title:
        sid.polarity_scores(title)['neu'])

    df.CompoundScoreText = df.Text.apply(lambda text:
        sid.polarity_scores(text)['compound'])

    df.PositveScoreText = df.Text.apply(lambda text:
        sid.polarity_scores(text)['pos'])

    df.NegativeScoreText = df.Text.apply(lambda text:
        sid.polarity_scores(text)['neg'])

    df.NeutralScoreText = df.Text.apply(lambda text:
        sid.polarity_scores(text)['neu'])
    
    df.titleLeans = df.CompoundScoreTitle.apply(lambda score:
                    'Negative' if score < 0 
                     else('Positive' if score > 0
                          else('Neutral')))
                        
    df.textLeans = df.CompoundScoreText.apply(lambda score:
                        'Negative' if score < 0 
                         else('Positive' if score > 0
                              else('Neutral')))
    
    df.CompoundAvg = (df.CompoundScoreTitle + df.CompoundScoreText)/2
    
    return df  

In [2]:
def get_timeframe(df):
    start = df.Date[len(df)-1][0:10]
    stop = df.Date[0][0:10]
    return(start)